# Importaciones

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Importamos la libreria para realizar la conexión con la base de datos
from sqlalchemy import create_engine

#Libreria para el manejo de datos
import pandas as pd
from datetime import datetime
import dateutil.relativedelta
import random

#Libreria para la descripción de las variables
import matplotlib.pyplot as plt

#Libreria para realizar los conjuntos de modelamiento, validación y pruebas
import numpy as np 
from sklearn.model_selection import train_test_split

#Libreria para utilizar la técnica de árboles extremadamente aleatorios
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

#LIbreria para guardar el entrenamiento de los extra trees
from joblib import dump, load

#Para realizar las redes neuronales
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, InputLayer, LSTM, Bidirectional
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.regularizers import l2
from keras.models import load_model

#Libreria para realizar comparaciónes
from keras import backend as K

Using TensorFlow backend.


# Carga de datos

Descargamos el archivo de datos desde la página http://www.sui.gov.co/web/datos-abiertos/energia/energia-formato-5-informacion-de-transformadores donde se encuentran el archivos de datos particular de enero de 2019 en el siguiente enlace http://www.sui.gov.co/datosAbiertos/Energia_442_M_4_2019.csv.zip. 

Es de importancia aclarar que se realizo la descarga de los datos de manera manual, debido a que la página web tiene protección para descargas automáticas.

## Funciones

In [ ]:
def df_aleatorios(df, cantidad):

  lenght = len(df) -1
  aleatorios = [ random.randint(0,lenght) for _ in range(cantidad)]
  df = df[aleatorios]

  return df

In [ ]:
# Funcion de lectura de los datos
def df_read():
    
        #Lectura del archivo a dataframe
        dataframe = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TFM/IA/formato5_520.zip',
                                header=0)
        
        #Convertimos las fecha a tipo fecha
        dataframe['fecha'] = pd.to_datetime(dataframe['fecha'])
        
        dataframe['mes'] = dataframe['fecha'].dt.month

        dataframe = dataframe.sort_values(by=['fecha'], ascending=False)
        
        #Convertimos los valores S y N en booleanos
        dic = {'S': True, 'N': False}
        
        dataframe['propiedad'] = dataframe['propiedad'].map(dic)
        
        print('Archivo cargado ',dataframe.shape)
        
        return dataframe

In [ ]:
# Función traer códigos de todos los transformadores
def df_tranformadores(df):
    
    unique_transformators = df['id_transformador'].unique()
    
    print('Transformadores unicos ',len(unique_transformators))
    
    return unique_transformators

In [ ]:
# Funcion para extraer dataframe de un transformador
def df_transformador(df, transformador):

    df_tr = df[(df['id_transformador'] == transformador)]

    del df_tr['id_transformador']
    
    return df_tr

In [ ]:
def df_salida(df, fecha):
    
    df_out = df[(df['fecha'] == fecha)]

    df_out = df_out[['min_catastrofes','min_expansion','min_infraestructura',
                     'min_inpne','min_ipne','min_limitacion','min_racionamiento', 
                     'min_seguridad', 'min_strstn', 'min_subestaciones', 'min_terrorismo', 
                     'min_usuario', 'min_zonasespeciales']]

    return df_out  

In [ ]:
# Función traer datos de entrada
def df_entrada_tiempo(df, fecha, espacio_tiempo):
    
    df_in = df[(df['fecha'] >= (fecha - dateutil.relativedelta.relativedelta(months=espacio_tiempo) ) )  & 
               (df['fecha'] < fecha)]

    del df_in['fecha']
    
    return df_in

In [ ]:
def time_print(sec):

       days = int(sec / 86400)
       sec -= 86400*days

       hrs = int(sec / 3600)
       sec -= 3600*hrs

       mins = int(sec / 60)
       sec -= 60*mins

       print('Duración ',days, ':', hrs, ':', mins, ':', sec)

In [ ]:
from time import time

# Función para entrenamiento de una fecha particular
# Con todos los transformadores
def df_conjunto_tiempo(fecha, espacio_tiempo, pruebas = 0):
    
    print('Leyendo el archivo de datos')
    
    df = df_read()
    
    print('Extrayendo información transformadores')
    
    transformador = df_tranformadores(df)
    
    #Limitación para las pruebas
    if pruebas != 0:
      print('Ejecutando Pruebas')
      transformador = df_aleatorios(transformador, pruebas)
    
    #Conjuntos
    X = []
    y = []

    #Lista de fechas
    # Rango de fechas años
    date_list = [fecha - dateutil.relativedelta.relativedelta(months=x) for x in range(108-espacio_tiempo)]

    #date_list = np.array(date_list)
    
    tiempo_inicial = time() 

    print('Cargando ',len(transformador))

    # Recorremos todos los transformadores
    # Tomamos conjuntos de entrada y predicción
    for t in transformador: 
        
        '''if (index * 100 / len(transformador)) % 10 == 0:
            print('Revisando el registro ',index, ' correspondiente al ',
                  index*100 / len(transformador),'% de ',len(transformador), ' registros')'''


        #transformator_dates = df_aleatorios(date_list, 18) #Para pruebas
        transformator_dates = date_list
        
        #Seleccianamos el conjunto de registro del tranformador
        df_tr = df_transformador(df, t)

        #Realizamos la extracción de conjunto para determianda fecha
        for date in transformator_dates:
        
            y1 = df_salida(df_tr, date)

            if not y1.empty:

              X1 = df_entrada_tiempo(df_tr, date, espacio_tiempo)

              if len(X1) == espacio_tiempo:

                      X.append(np.array(X1))
                      y.append(y1.values[0])
                
    time_print(time()  - tiempo_inicial)
    print('Cargado un total de: ',len(X),' datos')
        
    return np.array(X), np.array(y)

## Aplicación

In [ ]:
#X, y = df_conjunto_tiempo(datetime(2019,2,1), 12, 0)

In [ ]:
#dump(X, '/content/drive/My Drive/Colab Notebooks/TFM/IA/X_prueba.joblib.z')
#dump(y, '/content/drive/My Drive/Colab Notebooks/TFM/IA/y_prueba.joblib.z')

In [ ]:
X = load('/content/drive/My Drive/Colab Notebooks/TFM/IA/X_prueba.joblib.z')
y = load('/content/drive/My Drive/Colab Notebooks/TFM/IA/y_prueba.joblib.z')

# Determinación de conjunto de modelamiento, validación y pruebas

In [ ]:
# Cojunto de pruebas
train_x, test_x, train_y, test_y = train_test_split(X, y, 
                                                    test_size=0.2, random_state=2)

In [ ]:
# Cojunto de pruebas
train_x, validation_x, train_y, validation_y = train_test_split(train_x, train_y, 
                                                    test_size=0.2, random_state=4)

In [ ]:
print('Tamaño cojunto de entrenamiento final : ', len(train_x), len(train_y))
print('Tamaño cojunto de validación final : ', len(validation_x), len(validation_y))
print('Tamaño cojunto de pruebas final : ', len(test_x), len(test_y))

Tamaño cojunto de entrenamiento final :  637913 637913
Tamaño cojunto de validación final :  159479 159479
Tamaño cojunto de pruebas final :  199349 199349


# Redes Neuronales Profundas

In [ ]:
epochs = 1000
batch = 512

## Funciones

In [ ]:
# Función Input
def input_layer(model, size_x, size_y = None, size_z = None):
    
    if size_y == None:
        
        model.add( InputLayer( input_shape=(size_x, ) ) )
        
    elif size_z == None:
    
        model.add( InputLayer( input_shape=(size_x,size_y) ) )
        
    else:
        
        model.add( InputLayer( input_shape=(size_x,size_y, size_z) ) )
    
    model.add( BatchNormalization() )
    
    return model

In [ ]:
#Función Densa
def densa(model, neuronas= 256, activacion = 'relu', 
          regularizador = l2(0.001), dropout = 0.2):

    #Creación de una capa de 32 neuronas densas
    model.add(Dense(neuronas, activation=activacion, 
                    kernel_regularizer=regularizador,
                    bias_regularizer=regularizador))
  
    model.add(BatchNormalization())

    model.add(Dropout(dropout))

    return model

In [ ]:
#Función de resultado
def result_layer(model, y, activacion = 'linear'):
    
    model.add(Dense(y, activation=activacion))
    
    return model

In [ ]:
def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
def run_desing(model, name, batch, epochs, 
               train_x, train_y, 
               test_x, test_y, 
               optimizer = 'adam', loss = 'mean_squared_error'):
    
    metrics = ['mean_absolute_error','mean_squared_error',coeff_determination]
    model.name = name
    
    #Revisamos el modelo
    model.summary()
    
    #Compilación del modelo
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    #Con el objetivo de parar el entrenamiento si los valores no mejoran
    early_stop = EarlyStopping(monitor='val_loss', patience=epochs*0.05, verbose=1, 
                             min_delta=0.01, mode='auto')  

    #Con el objetivo de cambiar el learnfactor
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.001, patience=epochs*0.02, 
                                  mode='auto', verbose=1, min_delta=1e-4, 
                                  min_lr=0) 
    
    #Guardamos el mejor modelo
    checkpoint = ModelCheckpoint('/content/drive/My Drive/Colab Notebooks/TFM/IA/'+name+'.model', 
                                 monitor='val_loss', verbose=1, 
                                 save_best_only=True, mode='min')

    history = model.fit(train_x, train_y,
                        batch_size=batch,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(test_x, test_y),
                        callbacks=[early_stop, reduce_lr,
                                  checkpoint])
    
    model.save('/content/drive/My Drive/Colab Notebooks/TFM/IA/'+name+'.h5')

    dump([model,history,coeff_determination], '/content/drive/My Drive/Colab Notebooks/TFM/IA/'+name+'.joblib.z')
  
    return model, history

In [ ]:
def load_desing(name):

  #Cargamos desde disco
  model_complete = load('/content/drive/My Drive/Colab Notebooks/TFM/IA/'+name+'.joblib.lz4')

  #Cargamos el modelo, historia y variable especial
  model = model_complete[0]
  history = model_complete[1]
  coeff_determination = model_complete[2]

  return model, history

In [ ]:
def load_model_weights(model, name):

  model.load_weights('/content/drive/My Drive/Colab Notebooks/TFM/IA/'+name+'.model')

  return model

## Modelo

In [ ]:
def modelo_dnn():
    
    #Creación del modelo secuencial
    model = Sequential()
    
    function_act = 'relu'
    
    #Capa inicial
    model = input_layer(model, len(train_x[0]), len(train_x[0][0]))
    
    model.add(Flatten())
    
    model = densa(model, 1024, activacion = function_act)
    
    model = densa(model, 1024, activacion = function_act)
    
    model = densa(model, 512, activacion = function_act)
    
    model = densa(model, 512, activacion = function_act)
    
    model = densa(model, 256, activacion = function_act)
    
    model = result_layer(model, len(train_y[0]))
    
    return model

## Ejecución

In [26]:
model_dnn, history_dnn = run_desing(modelo_dnn(), 'modelo_dnn_pruebas', batch, epochs, 
                                    train_x, train_y, 
                                    validation_x, validation_y,
                                     'adam','huber_loss')

Model: "modelo_dnn_pruebas"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 12, 37)            148       
_________________________________________________________________
flatten_1 (Flatten)          (None, 444)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              455680    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)       

In [27]:
predict_y_dnn = model_dnn.predict(test_x)

# Red Neuronal Convolucional

## Funciones

In [ ]:
#Función convolucional
def convolutional(model, filtros = 256, kernel_x = 3,kernel_y = 3, 
                  stride = 1, activacion = 'relu', regularizador = l2(0.001), 
                  dropout = 0.2):

    #Capa convolucional
    model.add(Conv2D(filtros, kernel_size=(kernel_x, kernel_y),
                   activation=activacion,padding='same',
                   kernel_regularizer=regularizador,
                   bias_regularizer=regularizador))
    
    model.add(Dropout(dropout))

    return model

In [ ]:
#Función de pooling
def pooling(model, pooling_x = 3, pooling_y = 3, stride = 1,
           dropout = 0.2):

    #Capa de polling
    model.add(MaxPooling2D((pooling_x, pooling_y)))

    model.add(Dropout(dropout))
    
    model.add(BatchNormalization())

    return model

In [ ]:
def data_cnn(data):

  return data.reshape(len(data),len(data[0]),len(data[0][0]),1)

## Modelo

In [ ]:
def modelo_cnn():
    #Creación del modelo secuencial
    model = Sequential()
    
    function_act = 'relu'

    #Primera Capa convolucional
    
    model = input_layer(model, len(train_x[0]),len(train_x[0][0]),1)
    
    model = convolutional(model, 64, activacion = function_act)

    #model = pooling(model, 2, 2, dropout = 0)

    model = convolutional(model, 128, activacion = function_act)

    #model = pooling(model, 2, 2, dropout = 0)

    model = convolutional(model, 256, activacion = function_act)

    #model = pooling(model, 2, 2, dropout = 0)

    model = convolutional(model, 512, activacion = function_act)

    model = pooling(model, 2, 2, dropout = 0)

    model.add(Flatten())

    model = densa(model, 1024, activacion = function_act)
    model = densa(model, 1024, activacion = function_act)
    model = densa(model, 512, activacion = function_act)

    #Capa de salida
    model = result_layer(model, len(train_y[0]))

    return model

## Ejecución

In [32]:
#Transformación para CNN
train_x_cnn = data_cnn(train_x)
validation_x_cnn = data_cnn(validation_x)
test_x_cnn = data_cnn(test_x)

In [ ]:
model_cnn, history_cnn = run_desing(modelo_cnn(), 'modelo_cnn_pruebas', 
                                    batch, epochs, 
                                    train_x_cnn, train_y, 
                                    validation_x_cnn, validation_y,
                                     'adam','huber_loss')

Model: "modelo_cnn_pruebas"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_7 (Batch (None, 12, 37, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 37, 64)        640       
_________________________________________________________________
dropout_6 (Dropout)          (None, 12, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 37, 128)       73856     
_________________________________________________________________
dropout_7 (Dropout)          (None, 12, 37, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 37, 256)       295168    
_________________________________________________________________
dropout_8 (Dropout)          (None, 12, 37, 256)

In [ ]:
predict_y_cnn = model_cnn.predict(test_x_cnn)

# Red Neuronal Recurrente - LSTM

## Funciones

In [ ]:
def layer_lstm(model, neurons = 128, regularizer = l2(0.0001),
              dropout = 0.2, return_sequences = True):
    
    model.add(LSTM(neurons, return_sequences=return_sequences,
                   kernel_regularizer=regularizer,
                   bias_regularizer=regularizer, dropout=dropout,
                   recurrent_dropout=dropout))

    model.add(BatchNormalization())
    
    return model

## Modelo

In [ ]:
def model_lstm():
    
    #Creación del modelo secuencial
    model = Sequential()
    
    function_act = 'relu'

    #Primera Capa convolucional
    #Capa inicial
    model = input_layer(model, len(train_x[0]), len(train_x[0][0]) )
    
    model = layer_lstm(model, 128)
    
    model = layer_lstm(model, 256)
    
    model = layer_lstm(model, 128, return_sequences=False)
    
    #Capa de salida
    model = result_layer(model, len(train_y[0]))

    return model

## Ejecución

In [ ]:
model_lstm, history_lstm = run_desing(model_lstm(), 'modelo_lstm_pruebas', 
                                      batch, epochs,
                                      train_x, train_y, 
                                      validation_x, validation_y,
                                     'adam','huber_loss')

In [ ]:
predict_y_lstm = model_lstm.predict(test_x)

# Modelo red neuronal convolucional bidireccional con LSTM

## Función

In [ ]:
def layer_bi_lstm(model, neurons = 128, regularizer = l2(0.0001),
              dropout = 0.2, return_sequences = True):
    
    model.add(Bidirectional(LSTM(neurons, return_sequences=return_sequences,
                                 kernel_regularizer=regularizer,
                                 bias_regularizer=regularizer, dropout=dropout,
                                 recurrent_dropout=dropout)))

    model.add(BatchNormalization())
    
    return model

In [ ]:
#Función convolucional
def convolutional1D(model, filtros = 256, kernel_x = 3, 
                  stride = 1, activacion = 'relu', regularizador = l2(0.001), 
                  dropout = 0.2):

    #Capa convolucional
    model.add(Conv1D(filtros, kernel_size=kernel_x,
                   activation=activacion,padding='same',
                   kernel_regularizer=regularizador,
                   bias_regularizer=regularizador))
    
    model.add(Dropout(dropout))

    return model

In [ ]:
#Función de pooling
def pooling1D(model, pooling_x = 3, dropout = 0.2):

    #Capa de polling
    model.add(MaxPooling1D(pooling_x))

    model.add(Dropout(dropout))
    
    model.add(BatchNormalization())

    return model

## Modelo

In [ ]:
def model_bi_lstm():
    
    #Creación del modelo secuencial
    model = Sequential()
    
    function_act = 'relu'

    #Primera Capa convolucional
    #Capa inicial
    model = input_layer(model, len(train_x[0]),len(train_x[0][0]))
    
    model = convolutional1D(model, 64, activacion = function_act)

    model = pooling1D(model, 2, dropout = 0)
    
    model = layer_bi_lstm(model, 128)
    
    model = layer_bi_lstm(model, 128, return_sequences=False)
    
    model = densa(model, 64, activacion = function_act)
    
    #Capa de salida
    model = result_layer(model, len(train_y[0]))

    return model

## Ejecución

In [ ]:
model_bi_lstm, history_bi_lstm = run_desing(model_bi_lstm(), 
                                            'modelo_bilstm_pruebas_1c_1d', 
                                            batch, epochs, 
                                            train_x, train_y, 
                                            validation_x, validation_y,
                                            'adam','huber_loss')

In [ ]:
predict_y_bi_lstm = model_bi_lstm.predict(test_x)

# Árbol extremadamente aleatorio

## Funciones

In [ ]:
def ajustar_entrada(original):
    return np.reshape(original, (original.shape[0], original.shape[1]*original.shape[2]))

## Ejecución

In [ ]:
model_extra = ExtraTreesRegressor(n_estimators=350, random_state=0)

In [ ]:
history_extra = model_extra.fit(ajustar_entrada(train_x), train_y)

In [ ]:
dump(model_extra, '/content/drive/My Drive/Colab Notebooks/TFM/IA/model_extra_pruebas.joblib.z')

In [ ]:
predict_y_extra = model_extra.predict(ajustar_entrada(test_x))

## Metrica

In [ ]:
mean_squared_error(test_y,predict_y_extra)

In [ ]:
mean_absolute_error(test_y,predict_y_extra)

In [ ]:
r2_score(test_y,predict_y_extra)

# Comparación

## Funciones

In [ ]:
def all_metrics (test_y, predict_y, name):
    
    mse = mean_squared_error(test_y,predict_y)
    
    print('Para la técnica ',name, ' da un error cuadratico medio de ', mse)

    rmse = math.sqrt(mse)
    
    print('Para la técnica ',name, ' da una raíz del error cuadrático medio de ', mse)
    
    mae = mean_absolute_error(test_y,predict_y)
    
    print('Para la técnica ',name, ' da un error absoluto medio de ', mae)
    
    r2 = r2_score(test_y,predict_y)
    
    print('Para la técnica ',name, ' da un r cuadrado de ', r2*100, ' %')

    return [name, mse, rmse, mae, r2]

In [ ]:
#Ayuda a plotear una metrica para varias redes
# Entrada
#Tensor de histories
#Tensor de nombres
#Nombre de la metrica
# Salida
#No da salida
# Resultado
#Gráfico Ploteado
def plot_compare(histories, names, metric):

    #realizamos nobres para traer valores
    train_metric = metric
    test_metric = 'val_'+metric

    #Colocamos el tituo
    plt.title(metric)

    #Recorremos todas las historias
    for history in histories:
        color = np.random.rand(3,)
        plt.plot(history.history[train_metric], c=color)
        plt.plot(history.history[test_metric], 'r--', c=color)

    plt.ylabel(metric)
    plt.xlabel('Epoch')

    legend = []

    for name in names:
        legend.append('Entrenamiento ' + name)
        legend.append('Evaluación ' + name)

    plt.legend(legend,
               loc='lower right')

    plt.figure(figsize=(20,11))
    
    plt.show()

In [ ]:
def load_models(path, test_x):

  model = load_model('/content/drive/My Drive/Colab Notebooks/TFM/IA/'+path+'_pruebas.h5',
                     {"coeff_determination":coeff_determination})

  predict_y = model.predict(test_x)

  return model, predict_y

## Funciones para guardar los modelos

In [ ]:
def save_model(model,history,coeff_determination,name):
  dump([model,history,coeff_determination], 
       '/content/drive/My Drive/Colab Notebooks/TFM/IA/'+name+'.joblib.z')

In [ ]:
#save_model(model_dnn,history_dnn,coeff_determination,'model_dnn_pruebas2')

In [ ]:
#save_model(model_cnn,history_cnn,coeff_determination,'model_cnn_pruebas2')

In [ ]:
#save_model(model_lstm,history_lstm,coeff_determination,'model_lstm_pruebas2')

In [ ]:
#save_model(model_bi_lstm,history_bi_lstm,coeff_determination,'model_bi_lstm_pruebas2')

## Funciones para cargar nuevamente los modelos

In [ ]:
#model_dnn, predict_y_dnn = load_models('modelo_dnn_pruebas',test_x)

In [ ]:
#model_cnn, predict_y_cnn = load_models('modelo_cnn_pruebas',test_x_cnn)

In [ ]:
#model_lstm, predict_y_lstm = load_models('modelo_lstm_pruebas',test_x)

In [ ]:
#model_bi_lstm, predict_bi_lstm = load_models('modelo_bilstm_pruebas',test_x)

In [ ]:
#model_extra = load('/content/drive/My Drive/Colab Notebooks/TFM/IA/model_extra_pruebas.joblib.z')


## Metricas establecidas

In [ ]:
metrics_dnn = all_metrics(test_y, predict_y_dnn, 'Red Neuronal Profunda')

In [ ]:
metrics_cnn = all_metrics(test_y, predict_y_cnn, 'Red Neuronal Convolucional')

In [ ]:
metrics_lstm = all_metrics(test_y, predict_y_lstm, 'Red Neuronal LSTM')

In [ ]:
metrics_cblstmd = all_metrics(test_y, predict_y_bi_lstm, 'Convolucional Bi-Direccional LSTM Profunda')

In [ ]:
metrics_extra = all_metrics(test_y, predict_y_extra, 'Árboles extremadamente aleatorios')

In [ ]:
df_metrics = pd.DataFrame(columns=['Nombre','MSE','RMSE','MAE','R2'])
df_metrics.loc[df_metrics.shape[0]] = metrics_dnn
df_metrics.loc[df_metrics.shape[0]] = metrics_cnn
df_metrics.loc[df_metrics.shape[0]] = metrics_lstm
df_metrics.loc[df_metrics.shape[0]] = metrics_cblstmd
df_metrics.loc[df_metrics.shape[0]] = metrics_extra
df_metrics.set_index('Nombre', inplace=True)
df_metrics

## Gráficas comparativas DNN vs CNN vs LSTM

In [ ]:
#Gráficas comparativas de los diferentes resultados
histories = [history_dnn, history_lstm, history_bi_lstm, history_cnn]
names = ['Profunda', 'Recurrente LSTM', 'NNCRLSTMD', 'Convolucional',]

In [ ]:
#Loss Function
plot_compare(histories, names, 'loss')

In [ ]:
#Error medio absoluto
plot_compare(histories, names, 'mean_absolute_error')

In [ ]:
#Error cuadratico medio
plot_compare(histories, names, 'mean_squared_error')

In [ ]:
#Coeff
plot_compare(histories, names, 'coeff_determination')